# Algorithms Tutorial
Once we have data for GST, there are several *algorithms* we can run on it to produce tomographic estimates. Depending on the amount of data you have, and time available for running Gate Set Tomography, one algorithm may be preferable over the others.

Currently, `pygsti` provides support for the following GST algorithms:

* **Linear Gate Set Tomography (LGST)**: Uses short gate sequences to quickly compute a rough (low accuracy) estimate of a gate set by linear inversion.

* **Extended Linear Gate Set Tomography (eLGST or EXLGST)**: Minimizes the sub-of-squared errors between independent LGST estimates and the estimates obtained from a single gate set to find a best-estimate gate set.  This is typically done in an interative fashion, using LGST estimates for longer and longer sequences. 

* **Minimum-$\chi^2$ Gate Set Tomography (MC2GST)**: Minimizes the $\chi^{2}$ statistic of the data frequencies and gate set probabilities to find a best-estimate gate set.  Typically done in an interative fashion, using successively larger sets of longer and longer gate sequences. 

* **Maximum-Likelihood Gate Set Tomography (MLGST)**: Maximizes the log-likelihood statistic of the data frequencies and gate set probabilities to find a best-estimate gate set.  Typically done in an interative fashion similar to MC2GST.  This maximum likelihood estimation (MLE) is very well-motivated from a statistics standpoint and should be the **most accurate among the algorithms**. 

If you're curious, the implementation of the algorithms for LGST, EXLGST, MC2GST, and MLGST may be found in the `pygsti.algorithms.core` module.  In this tutorial, we'll show how to invoke each of these algorithms.

###  Setup
The ingredients needed to as input to the GST algorithms are:
- a "target" `GateSet` which defines the desired gates.  This gate set is used by LGST to specify the various gate, state preparation, POVM effect, and SPAM labels, as well as to provide an initial guess for the *gauge* degrees of freedom.
- a `DataSet` containing the data that GST attempts to fit using the probabilities generated by a single `GateSet`.  This data set must at least contain the data for the gate sequences required by the algorithm that is chosen.
- for EXLGST, MC2GST, and MLGST, a list-of-lists of `GateString` objects, which specify which gate strings are used during each iteration of the algorithm (the length of the top-level list defines the number of interations).  Note that which gate strings are included in these lists is different for EXLGST than it is for MC2GST and MLGST.

In [1]:
import pygsti
import json

In [2]:
#Load target gateset, dataset, and list of fiducial gate strings.
#In this case we load directly from files created in past tutorials

gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)
dsLowCounts = pygsti.io.load_dataset("tutorial_files/Example_Dataset_LowCnts.txt", cache=True)
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")

depol_gateset = gs_target.depolarize(gate_noise=0.1)

#Could also load a fiducial dictionary file like this:
#fiducialDict = GST.load_gatestring_dict("Example_FiducialList.txt")
#fiducialList = fiducialDict.values() #all we really need are the fiducial strings themselves

print("Loaded target gateset with gate labels: ", gs_target.gates.keys())
print("Loaded fiducial list of length: ", len(fiducialList))
print("Loaded dataset of length: ", len(ds))

Loading tutorial_files/Example_Dataset_LowCnts.txt: 100%
Writing cache file (to speed future loads): tutorial_files/Example_Dataset_LowCnts.txt.cache
Loaded target gateset with gate labels:  odict_keys(['Gi', 'Gx', 'Gy'])
Loaded fiducial list of length:  6
Loaded dataset of length:  2737


### Using LGST to get an initial estimate

An important and distinguising property of the LGST algorithm is that it does *not* require an initial-guess `GateSet` as an input.  It uses linear inversion and short sequences to obtain a rough gate set estimate.  As such, it is very common to use the LGST estimate as the initial-guess starting point for more advanced forms of GST.

In [3]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds

#create "spam specs" from the list of fiducial gate strings.  See the gate strings tutorial for more info.
specs = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)

#run LGST
gs_lgst = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = pygsti.optimize_gauge(gs_lgst, "target", targetGateset=gs_target)

#Contract the result to CPTP, guaranteeing that the gates are CPTP
gs_clgst = pygsti.contract(gs_lgst_after_gauge_opt, "CPTP")

--- LGST ---


In [4]:
print(gs_lgst)

rho0 =    0.7094  -0.0205   0.0230   0.7544


E0 =    0.6877   0.0052  -0.0021  -0.6498


Gi = 
   1.0065  -0.0005  -0.0018   0.0055
  -0.0059   0.9258   0.0528  -0.0169
   0.0422  -0.0172   0.9029   0.0212
  -0.0078   0.0118   0.0219   0.9076


Gx = 
   0.9970  -0.0073  -0.0088   0.0003
   0.0056   0.9077   0.0232  -0.0016
   0.0146   0.0163   0.0013  -1.0031
  -0.0695  -0.0060   0.8032   0.0079


Gy = 
   1.0040   0.0074  -0.0055  -0.0005
  -0.0377  -0.0225  -0.0012   0.9963
   0.0189   0.0094   0.8877  -0.0266
  -0.0679  -0.8058  -0.0250   0.0178





### Extended LGST (eLGST or EXLGST)
EXLGST requires a list-of-lists of gate strings, one per iteration.  The elements of these lists are typically repetitions of short "germ" strings such that the final strings does not exceed some maximum length.  We created such lists in the gate string tutorial.  Now, we just load these lists from the text files they were saved in.

In [5]:
#Get rho and E specifiers, needed by LGST
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
elgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_eLGSTlist%d.txt" % l) for l in maxLengthList]
           
#run EXLGST.  The result, gs_exlgst, is a GateSet containing the estimated quantities
gs_exlgst = pygsti.do_iterative_exlgst(ds, gs_clgst, specs, elgstListOfLists, targetGateset=gs_target,
                                  svdTruncateTo=4, verbosity=2)

    
  --- Iterative eLGST:  Iter 9 of 9 ; 84 gate strings ---: 


### Minimum-$\chi^2$ GST (MC2GST)
MC2GST and MLGST also require a list-of-lists of gate strings, one per iteration.  However, the elements of these lists are typically repetitions of short "germ" strings *sandwiched between fiducial strings* such that the repeated-germ part of the string does not exceed some maximum length.  We created such lists in the gate string tutorial.  Now, we just load these lists from the text files they were saved in.

In [6]:
#Get lists of gate strings for successive iterations of LSGST to use
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
lsgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l) for l in maxLengthList]
  
#run MC2GST.  The result is a GateSet containing the estimated quantities
gs_mc2 = pygsti.do_iterative_mc2gst(ds, gs_clgst, lsgstListOfLists, verbosity=2)

    
  --- Iterative MC2GST: Iter 0 of 9  92 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 40.9237 (92 data params - 40 model params = expected mean of 52; p-value = 0.866036)
    
  --- Iterative MC2GST: Iter 1 of 9  92 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 40.9237 (92 data params - 40 model params = expected mean of 52; p-value = 0.866036)
    
  --- Iterative MC2GST: Iter 2 of 9  168 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 124.56 (168 data params - 40 model params = expected mean of 128; p-value = 0.569542)
    
  --- Iterative MC2GST: Iter 3 of 9  441 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 424.143 (441 data params - 40 model params = expected mean of 401; p-value = 0.204567)
    
  --- Iterative MC2GST: Iter 4 of 9  817 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 772.113 (817 data params - 40 model params = expected mean of 777; p-value = 0.542725)
    
  --- Iterative M

In [7]:
#Write the resulting EXLGST and MC2GST results to gate set text files for later reference.
pygsti.io.write_gateset(gs_exlgst, "tutorial_files/Example_eLGST_Gateset.txt","# Example result from running eLGST")
pygsti.io.write_gateset(gs_mc2,  "tutorial_files/Example_MC2GST_Gateset.txt","# Example result from running MC2GST")

In [8]:
#Run MC2GST again but use a DataSet with a lower number of counts 
gs_mc2_lowcnts = pygsti.do_iterative_mc2gst(dsLowCounts, gs_clgst, lsgstListOfLists, verbosity=2)

    
  --- Iterative MC2GST: Iter 0 of 9  92 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 64.0311 (92 data params - 40 model params = expected mean of 52; p-value = 0.122289)
    
  --- Iterative MC2GST: Iter 1 of 9  92 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 64.0311 (92 data params - 40 model params = expected mean of 52; p-value = 0.122289)
    
  --- Iterative MC2GST: Iter 2 of 9  168 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 161.612 (168 data params - 40 model params = expected mean of 128; p-value = 0.0237629)
    
  --- Iterative MC2GST: Iter 3 of 9  441 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 427.539 (441 data params - 40 model params = expected mean of 401; p-value = 0.173449)
    
  --- Iterative MC2GST: Iter 4 of 9  817 gate strings ---: 
--- Minimum Chi^2 GST ---
    Sum of Chi^2 = 770.758 (817 data params - 40 model params = expected mean of 777; p-value = 0.556396)
    
  --- Iterative

### Maximum Likelihood GST (MLGST)
Executing MLGST is very similar to MC2GST: the same gate string lists can be used and calling syntax is nearly identitcal.

In [9]:
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
lsgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l) for l in maxLengthList] 

#run MLGST.  The result is a GateSet containing the estimated quantities
gs_mle = pygsti.do_iterative_mlgst(ds, gs_clgst, lsgstListOfLists, verbosity=2)

    
  --- Iterative MLGST: Iter 0 of 9  92 gate strings ---: 
    
  --- Iterative MLGST: Iter 1 of 9  92 gate strings ---: 
    
  --- Iterative MLGST: Iter 2 of 9  168 gate strings ---: 
    
  --- Iterative MLGST: Iter 3 of 9  441 gate strings ---: 
    
  --- Iterative MLGST: Iter 4 of 9  817 gate strings ---: 
    
  --- Iterative MLGST: Iter 5 of 9  1201 gate strings ---: 
    
  --- Iterative MLGST: Iter 6 of 9  1585 gate strings ---: 
    
  --- Iterative MLGST: Iter 7 of 9  1969 gate strings ---: 
    
  --- Iterative MLGST: Iter 8 of 9  2353 gate strings ---: 
    
  --- Iterative MLGST: Iter 9 of 9  2737 gate strings ---: 
    --- Last Iteration: switching to ML objective ---
    
--- MLGST ---
    Least squares msg = The relative error between two consecutive iterates is at most 0.000001; flag =2
    Maximum log(L) = 1400.49 below upper bound of -4.60013e+06
      2*Delta(log(L)) = 2800.99 (2737 data params - 40 model params = expected mean of 2697; p-value = 0.0796974)
  

In [10]:
#Run MLGST again but use a DataSet with a lower number of counts 
gs_mle_lowcnts = pygsti.do_iterative_mlgst(dsLowCounts, gs_clgst, lsgstListOfLists, verbosity=2)

    
  --- Iterative MLGST: Iter 0 of 9  92 gate strings ---: 
    
  --- Iterative MLGST: Iter 1 of 9  92 gate strings ---: 
    
  --- Iterative MLGST: Iter 2 of 9  168 gate strings ---: 
    
  --- Iterative MLGST: Iter 3 of 9  441 gate strings ---: 
    
  --- Iterative MLGST: Iter 4 of 9  817 gate strings ---: 
    
  --- Iterative MLGST: Iter 5 of 9  1201 gate strings ---: 
    
  --- Iterative MLGST: Iter 6 of 9  1585 gate strings ---: 
    
  --- Iterative MLGST: Iter 7 of 9  1969 gate strings ---: 
    
  --- Iterative MLGST: Iter 8 of 9  2353 gate strings ---: 
    
  --- Iterative MLGST: Iter 9 of 9  2737 gate strings ---: 
    --- Last Iteration: switching to ML objective ---
    
--- MLGST ---
    Least squares msg = Both actual and predicted relative reductions in the sum of squares
  are at most 0.000001; flag =1
    Maximum log(L) = 1357.02 below upper bound of -228670
      2*Delta(log(L)) = 2714.04 (2737 data params - 40 model params = expected mean of 2697; p-value =

## Compare MLGST with MC2GST

Both MLGST and MC2GST use a $\chi^{2}$ optimization procedure for all but the final iteration. For the last set of gatestrings (the last iteration), MLGST uses a maximum likelihood estimation. Below, we show how close the two estimates are to one another.  First, we optimize the gauge so the estimated gates are as close to the target gates as the gauge degrees of freedom allow.

In [11]:
# We optimize over the gate set gauge
gs_mle         = pygsti.optimize_gauge(gs_mle,"target",targetGateset=depol_gateset)
gs_mle_lowcnts = pygsti.optimize_gauge(gs_mle_lowcnts,"target",targetGateset=depol_gateset)
gs_mc2         = pygsti.optimize_gauge(gs_mc2,"target",targetGateset=depol_gateset)
gs_mc2_lowcnts = pygsti.optimize_gauge(gs_mc2_lowcnts,"target",targetGateset=depol_gateset)

In [12]:
print("Frobenius diff btwn MLGST  and datagen = {0}".format(round(gs_mle.frobeniusdist(depol_gateset), 6)))
print("Frobenius diff btwn MC2GST and datagen = {0}".format(round(gs_mc2.frobeniusdist(depol_gateset), 6)))
print("Frobenius diff btwn MLGST  and LGST    = {0}".format(round(gs_mle.frobeniusdist(gs_clgst), 6)))
print("Frobenius diff btwn MLGST  and MC2GST  = {0}".format(round(gs_mle.frobeniusdist(gs_mc2), 6)))
print("Chi^2 ( MC2GST ) = {0}".format(round(pygsti.chi2(ds, gs_mc2, lsgstListOfLists[-1]), 4)))
print("Chi^2 ( MLGST )  = {0}".format(round(pygsti.chi2(ds, gs_mle, lsgstListOfLists[-1] ), 4)))
print("LogL  ( MC2GST ) = {0}".format(round(pygsti.logl(gs_mc2, ds, lsgstListOfLists[-1]), 4)))
print("LogL  ( MLGST )  = {0}".format(round(pygsti.logl(gs_mle, ds, lsgstListOfLists[-1]), 4)))

Frobenius diff btwn MLGST  and datagen = 0.110837
Frobenius diff btwn MC2GST and datagen = 0.110781
Frobenius diff btwn MLGST  and LGST    = 0.01636
Frobenius diff btwn MLGST  and MC2GST  = 0.000496
Chi^2 ( MC2GST ) = 2797.8679
Chi^2 ( MLGST )  = 2798.5944
LogL  ( MC2GST ) = -4601530.8815
LogL  ( MLGST )  = -4601530.5238


Notice that, as expected, the MC2GST estimate has a *slightly* lower $\chi^{2}$ score than the MLGST estimate, and the MLGST estimate has a *slightly* higher loglikelihood than the MC2GST estimate. In addition, _both_ are close (in terms of the Frobenius difference) to the depolarized gateset. Which is good - it means GST is giving us estimates which are close to the _true_ gateset used to generate the data.  Performing the same analysis with the low-count data shows larger differences between the two, which is expected since the $\chi^2$ and loglikelihood statistics are more similar at large $N$, that is, for large numbers of samples.

In [13]:
print("LOW COUNT DATA:")
print("Frobenius diff btwn MLGST  and datagen = {0}".format(round(gs_mle_lowcnts.frobeniusdist(depol_gateset), 6)))
print("Frobenius diff btwn MC2GST and datagen = {0}".format(round(gs_mc2_lowcnts.frobeniusdist(depol_gateset), 6)))
print("Frobenius diff btwn MLGST  and LGST    = {0}".format(round(gs_mle_lowcnts.frobeniusdist(gs_clgst), 6)))
print("Frobenius diff btwn MLGST  and MC2GST  = {0}".format(round(gs_mle_lowcnts.frobeniusdist(gs_mc2), 6)))
print("Chi^2 ( MC2GST )  = {0}".format(round(pygsti.chi2(dsLowCounts, gs_mc2_lowcnts, lsgstListOfLists[-1]), 4)))
print("Chi^2 ( MLGST )   = {0}".format(round(pygsti.chi2(dsLowCounts, gs_mle_lowcnts, lsgstListOfLists[-1] ), 4)))
print("LogL  ( MC2GST )  = {0}".format(round(pygsti.logl(gs_mc2_lowcnts, dsLowCounts, lsgstListOfLists[-1]), 4)))
print("LogL  ( MLGST )   = {0}".format(round(pygsti.logl(gs_mle_lowcnts, dsLowCounts, lsgstListOfLists[-1]), 4)))

LOW COUNT DATA:
Frobenius diff btwn MLGST  and datagen = 0.109914
Frobenius diff btwn MC2GST and datagen = 0.110838
Frobenius diff btwn MLGST  and LGST    = 0.016371
Frobenius diff btwn MLGST  and MC2GST  = 0.015121
Chi^2 ( MC2GST )  = 2671.4486
Chi^2 ( MLGST )   = 2685.8084
LogL  ( MC2GST )  = -230034.3767
LogL  ( MLGST )   = -230027.2495
